In [70]:
import pandas as pd
import numpy as np
import re
import string
import nltk        
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report 
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier

In [71]:
df = pd.read_csv("consumer_complaints.csv",index_col=0)
df.head()

/var/folders/bl/4hy1jx015313sscgqhkmm6_c0000gn/T/ipykernel_39661/1960908724.py:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("consumer_complaints.csv",index_col=0)


,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
date_received,,,,,,,,,,,,,,,,,
08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [72]:
state_abbr = [
    'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
    'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
    'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
    'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
    'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
df = df[df['state'].isin(state_abbr)]

In [73]:
features = ['product', 'state', 'consumer_disputed?', 'submitted_via']
df = df.replace(['Yes', 'No'], [1, 0])
data = df[features]
data

,product,state,consumer_disputed?,submitted_via
date_received,,,,
08/30/2013,Mortgage,CA,1,Referral
08/30/2013,Mortgage,CA,1,Referral
08/30/2013,Credit reporting,NY,0,Postal mail
08/30/2013,Student loan,MD,1,Email
08/30/2013,Debt collection,GA,1,Web
...,...,...,...,...
03/23/2016,Mortgage,NJ,0,Web
03/31/2016,Mortgage,OH,0,Referral
03/08/2016,Mortgage,CO,0,Web


In [75]:
prod = pd.get_dummies(data['product'])
state = pd.get_dummies(data['state'])
disputed = pd.get_dummies(data['consumer_disputed?'])
sub = pd.get_dummies(data['submitted_via'])

In [76]:
dat = pd.concat([prod, state, sub], axis=1)

dat

,Bank account or service,Consumer Loan,Credit card,Credit reporting,Debt collection,Money transfers,Mortgage,Other financial service,Payday loan,Prepaid card,...,WA,WI,WV,WY,Email,Fax,Phone,Postal mail,Referral,Web
date_received,,,,,,,,,,,,,,,,,,,,,
08/30/2013,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
08/30/2013,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
08/30/2013,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
08/30/2013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
08/30/2013,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
03/23/2016,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
03/31/2016,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
03/08/2016,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [77]:
X = dat
y = pd.get_dummies(data['consumer_disputed?'])
X_tr, X_t, y_tr, y_t = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_tr, y_tr)
y_pred = classifier2.predict(X_t)

print("Classification Report:\n\n", classification_report(y_t,y_pred))

In [ ]:
class_names = ['credit_card','credit_reporting','debt_collection','mortgages_and_loans','retail_banking']
plt.figure(figsize=(10,10))
plot_confusion_matrix(rfc,X_test,y_test,cmap=plt.cm.Blues,normalize='true')
plt.xticks(rotation=90)
plt.title("Confusion Matrix")